# NLP RECSYS pipeline test




In [1]:
import com.johnsnowlabs.nlp.SparkNLP
import com.johnsnowlabs.nlp.annotator._
import com.johnsnowlabs.nlp.base._
import com.johnsnowlabs.ml.tensorflow.TensorflowBert
import org.apache.spark.ml.Pipeline
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions.{udf,to_timestamp}
import org.apache.spark.ml.feature.QuantileDiscretizer
import org.apache.spark.storage._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification._
import org.apache.spark.ml.linalg.DenseVector

// Imports below are used for customer ML Estimator
import org.apache.spark.ml.Estimator
import org.apache.spark.ml.param.{Param, ParamMap}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col, explode, udf}
import org.apache.spark.sql.types.{DataTypes, StructType}

val dataDir = sys.env("HOME") + "/recsys2020"

In [4]:
var trainDf = Seq(0 to 1000000:_*).toDF()
    .withColumn("tweet_id", monotonically_increasing_id)
    .withColumn("user_id", monotonically_increasing_id)
for (i <- 0 to 522)
{
    trainDf = trainDf.withColumn("f_" + i, rand)
}
   
trainDf.show()
trainDf.printSchema

In [19]:
trainDf.count()

100

In [5]:
// NLP Feature Engineering

// val doc = new DocumentAssembler()
//     .setInputCol("text")
//     .setOutputCol("document")
//     .setCleanupMode("shrink")
// val tok = new Tokenizer()
//     .setInputCols("document")
//     .setOutputCol("token")
//     .setContextChars(Array("(", ")", "?", "!"))
//     .setSplitChars(Array("-"))
//     .addException("New York")
//     .addException("e-mail")
// val bert = BertEmbeddings.pretrained(name="bert_multi_cased", lang="xx")
//       .setInputCols("document", "token")
//       .setOutputCol("embeddings")
//       .setPoolingLayer(0) // 0, -1, or -2
// val use = UniversalSentenceEncoder
//       .pretrained()
//       .setInputCols(Array("document"))
//       .setOutputCol("tweet_embeddings")
// val emb = new SentenceEmbeddings()
//       .setInputCols(Array("document", "embeddings"))
//       .setOutputCol("tweet_embeddings")
//       .setPoolingStrategy("AVERAGE")
val fin = new EmbeddingsFinisher()
      .setInputCols("tweet_embeddings")
      .setOutputCols("nested_tweet_embeddings")
      .setOutputAsVector(true)
      .setCleanAnnotations(false)

// There should be only 1 tweet embedding vector per tweet
// val toVec = new VectorAssembler()
//   .setInputCols(Array("nested_tweet_embeddings"))
//   .setOutputCol("tweet_embeddings_vector")
// val embSlice = new VectorSlicer()
//       .setInputCol("nested_tweet_embeddings")
//       .setOutputCol("finished_tweet_embeddings")
//       .setIndices(Array(0))

// TODO: add from below
// Remaining feature engineering


// Assemble the pipeline
val text_trans_pipeline = new Pipeline().setStages(Array(doc, use, fin))//, toVec, embSlice))

Error: not found: value doc (1620)Error: not found: value use (1625)

In [6]:
// can later be combined into one large pipeline
val intermediateDf = text_trans_pipeline.fit(trainDf).transform(trainDf)
// intermediateDf.select("nested_tweet_embeddings").toDF()
intermediateDf//.show()

Error: not found: value text_trans_pipeline (70)

In [7]:
// assumption: only one embedding is generated, thus explode just flattens the list
val transDf = intermediateDf.withColumn("embedding_features", explode('nested_tweet_embeddings))
transDf.show()

Error: not found: value intermediateDf (98)

In [8]:
transDf

Error: not found: value transDf (0)

In [9]:
val scaleAss = new VectorAssembler()
  .setInputCols(Array("arbraryIntFeature"))
  .setOutputCol("arbraryIntFeatureVector")

val scaler = new StandardScaler()
  .setInputCol("arbraryIntFeatureVector")
  .setOutputCol("scaledFeatureVector")
  .setWithStd(true)
  .setWithMean(false)

val ass = new VectorAssembler()
  .setInputCols(Array("binaryFeature", "arbraryIntFeature"))
  .setOutputCol("features")
  .

// You need to setProbabilityCol,  setPredictionCol and setRawPredictionCol
// otherwise you get a name conflict in the pipeline

val gbtT1 = new GBTClassifier()
  .setLabelCol("target1")
  .setFeaturesCol("features")
  .setProbabilityCol("prob1")
  .setPredictionCol("pred1")
  .setRawPredictionCol("rpred1")
  .setMaxIter(10)
  .setFeatureSubsetStrategy("auto")

val gbtT2 = new GBTClassifier()
  .setLabelCol("target2")
  .setFeaturesCol("features")
  .setProbabilityCol("prob2")
  .setPredictionCol("pred2")
  .setRawPredictionCol("rpred2")
  .setMaxIter(10)
  .setFeatureSubsetStrategy("auto")

val pred_pipeline = new Pipeline().setStages(Array(scaleAss, scaler, ass))

In [10]:
val finalDf = pred_pipeline.fit(trainDf).transform(trainDf).select("features")
finalDf

[features: vector]

In [11]:
trainDf.show(false)
finalDf.show(false)

+-----+------------+-------------+-----------------+------------------+-------+-------+-------+------+
|value|text        |binaryFeature|arbraryIntFeature|arrayStringFeature|target1|target2|tweedId|userId|
+-----+------------+-------------+-----------------+------------------+-------+-------+-------+------+
|0    |Hello World.|1            |0                |[]                |1      |1      |0      |0     |
|1    |Hello World.|0            |1                |[]                |1      |1      |1      |1     |
|2    |Hello World.|0            |2                |[]                |0      |0      |2      |2     |
|3    |Hello World.|1            |3                |[Tag1, Tag2]      |1      |1      |3      |3     |
|4    |Other       |0            |4                |[]                |0      |1      |4      |4     |
|5    |Hello World.|0            |5                |[]                |1      |1      |5      |5     |
|6    |Hello World.|1            |6                |[]                |1 

In [12]:
val vecToArray = udf( (xs: org.apache.spark.ml.linalg.DenseVector) => xs.toArray )
val elements = Array("f1", "f2")
val sqlExpr = elements.zipWithIndex.map{ case (alias, idx) => col("feat_arr").getItem(idx).as(alias) }

finalDf.withColumn("feat_arr", vecToArray(col("features"))).select(sqlExpr : _*).show


+---+----+
| f1|  f2|
+---+----+
|1.0| 0.0|
|0.0| 1.0|
|0.0| 2.0|
|1.0| 3.0|
|0.0| 4.0|
|0.0| 5.0|
|1.0| 6.0|
|1.0| 7.0|
|1.0| 8.0|
|0.0| 9.0|
|0.0|10.0|
|0.0|11.0|
|1.0|12.0|
|0.0|13.0|
|1.0|14.0|
|0.0|15.0|
|1.0|16.0|
|0.0|17.0|
|0.0|18.0|
|1.0|19.0|
+---+----+
only showing top 20 rows



In [13]:
finalDf.select("target1", "prob1", "pred1", "features").show(false)

In [14]:
finalDf.select("target2", "prob2", "pred2", "features").show(false)

In [15]:
finalDf

In [16]:
// only get relevant columns
val toArr: Any => Double = _.asInstanceOf[DenseVector].toArray(1)
val toArrUdf = udf(toArr)

finalDf.withColumn("output2", toArrUdf('prob2)).select("userId", "tweedId", "output2").show()

In [17]:
text_trans_pipeline.save(dataDir + "/pipeline_nlp.model")
pred_pipeline.save(dataDir + "/pipeline_ml.model")

In [18]:
class VectorLookupModel(override val uid: String = "VECTORLOOKUPKDSAFJKDSAJF") extends Transformer {

}

class ArrayVectorAggregator(override val uid: String = "ARRAYVECTORAGGREGATORKDSAFJKDSAJF") extends Estimator[VectorLookupModel] {

  // Transformer Params
  // Defining a Param requires 3 elements:
  //  - Param definVeition
  //  - Param getter method
  //  - Param setter method
  // (The getter and setter are technically not required, but they are nice standards to follow.)

  /**
   * Param for input array column name.
   */
  final val inputCol: Param[String] = new Param[String](this, "inputCol", "input column name")

  final def getInputCol: String = $(inputCol)

  final def setInputCol(value: String): VectorLookupModel = set(inputCol, value)

  /**
   * Param for lookup vector column name.
   */
  final val vectorCol: Param[String] = new Param[String](this, "outputCol", "output column name")

  final def getVectorCol: String = $(outputCol)

  final def setVectorCol(value: String): VectorLookupModel = set(outputCol, value)

  /**
   * Param for output column name.
   */
  final val outputCol: Param[String] = new Param[String](this, "outputCol", "output column name")

  final def getOutputCol: String = $(outputCol)

  final def setOutputCol(value: String): VectorLookupModel = set(outputCol, value)

  /**
   * This method implements the main transformation.
   * Its required semantics are fully defined by the method API: take a Dataset or DataFrame,
   * and return a DataFrame.
   *
   * Most Transformers are 1-to-1 row mappings which add one or more new columns and do not
   * remove any columns.  However, this restriction is not required.  This example does a flatMap,
   * so we could either (a) drop other columns or (b) keep other columns, making copies of values
   * in each row as it expands to multiple rows in the flatMap.  We do (a) for simplicity.
   */
  override def fit(dataset: Dataset[_]): VectorLookupModel = {
    // dataset.withColumn($(outputCol), explode(col($(inputCol))))
  }

  /**
   * Check transform validity and derive the output schema from the input schema.
   *
   * We check validity for interactions between parameters during `transformSchema` and
   * raise an exception if any parameter value is invalid. Parameter value checks which
   * do not depend on other parameters are handled by `Param.validate()`.
   *
   * Typical implementation should first conduct verification on schema change and parameter
   * validity, including complex parameter interaction checks.
   */
  override def transformSchema(schema: StructType): StructType = {
      val inputColType = schema.fields(schema.fieldIndex($(inputCol))).dataType.asInstanceOf[ArrayType];
      // Compute output type.
      // This is important to do correctly when plugging this Transformer into a Pipeline,
      // where downstream Pipeline stages may expect use this Transformer's output as their input.
    //   schema.add($(outputCol), inputColType.elementType)
  }

  /**
   * Creates a copy of this instance.
   * Requirements:
   *  - The copy must have the same UID.
   *  - The copy must have the same Params, with some possibly overwritten by the `extra`
   *    argument.
   *  - This should do a deep copy of any data members which are mutable.  That said,
   *    Transformers should generally be immutable (except for Params), so the `defaultCopy`
   *    method often suffices.
   * @param extra  Param values which will overwrite Params in the copy.
   */
  override def copy(extra: ParamMap): Estimator = defaultCopy(extra)
}
